In [48]:
import pandas as pd
from pathlib import Path

In [49]:
root = Path().resolve()
data_path = root / "data"
csv_file = data_path / "mantis_export_all.csv"

csv_df = pd.read_csv(csv_file)
filt = (csv_df['Description'].notna()) & (csv_df['Status'] == 'Closed')
csv_df = csv_df[filt]
csv_df.set_index('Id', inplace=True)
csv_df

,Project,Reporter,Assigned To,Priority,Severity,Category,Date Submitted,View Status,Updated,Summary,Status,Target Date,Description,Notes
Id,,,,,,,,,,,,,,
25594,Farley Maintenance,vkozel,vkozel,normal,minor,(No Category),01/15/25,public,01/15/25,FNP-30442: Sim A ARV M/A stations do not contr...,Feedback,NaN,"The ARV Controllers, PC-3371A/B/C do not contr...",@ylacombe 01/15/2025\nDid you monitor the AI p...
25576,Sequoyah Maintenance,cmerriman,cmerriman,normal,minor,General,01/10/25,public,01/15/25,ThunderView/IA/DCS Drawing showing 0.00 when v...,Feedback,NaN,Some ThunderView drawings are showing valves b...,@ylacombe 01/15/2025\nThe popup in the note ab...
25301,Grand Gulf Simulator Modernization ENT22215,tscarborough,jsudario,normal,minor,Panel Graphics,10/30/24,public,01/15/25,P669 panel NUMACS,Assigned,NaN,The Numac controllers on panel P669 do not hav...,@jsudario (private) 01/15/2025\nNote to self: ...
25524,Farley Maintenance,vkozel,vkozel,urgent,minor,(No Category),12/18/24,public,01/15/25,Need to simulate a leak in Seal Injection Pipi...,Resolved,NaN,To facilitate training on a LOCA outside conta...,@djackson 01/15/2025\nAttaching updated nodal\...
25571,Grand Gulf Simulator Modernization ENT22215,mellis,rfuller,high,major,THOR BOP,01/09/25,public,01/15/25,Condenser inleakage,Resolved,NaN,This was identified by LOR using the C25 LOR l...,@dofarrell 01/15/2025\nthe gist of this DR wit...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,TMI Rad Transport EXE09083,jtesmer,NaN,normal,minor,NaN,05/01/10,public,05/03/10,Data Request: Aux Bldg Ventilation system draw...,Closed,NaN,Attached are additional drawings of the Aux an...,NaN
131,TMI Rad Transport EXE09083,jtesmer,NaN,normal,minor,NaN,05/01/10,public,05/03/10,Create categories DR and Data Request,Closed,NaN,Create a data request category so that this sy...,NaN
16,Cooper NPP08045,lryan,dofarrell,low,minor,NaN,04/14/10,private,04/29/10,Cooper T & M Malf CS03 dtd 03/09/10,Closed,NaN,We have another work request for we would like...,@dofarrell 04/29/2010\nTalked to Don Dea - no ...


In [50]:

# csv_df["LLM_input"] = "[[Problem Title]]\n" + csv_df['Summary'] + "[[Problem Description]]\n" + csv_df['Description'] + "[[Notes and solution]]\n" + csv_df['Notes'] 
# l = csv_df["LLM_input"].to_list()
csv_to_llm_df = csv_df[['Summary','Project', 'Description', 'Notes']]
csv_to_llm_df
# csv_to_llm_df

,Summary,Project,Description,Notes
Id,,,,
25594,FNP-30442: Sim A ARV M/A stations do not contr...,Farley Maintenance,"The ARV Controllers, PC-3371A/B/C do not contr...",@ylacombe 01/15/2025\nDid you monitor the AI p...
25576,ThunderView/IA/DCS Drawing showing 0.00 when v...,Sequoyah Maintenance,Some ThunderView drawings are showing valves b...,@ylacombe 01/15/2025\nThe popup in the note ab...
25301,P669 panel NUMACS,Grand Gulf Simulator Modernization ENT22215,The Numac controllers on panel P669 do not hav...,@jsudario (private) 01/15/2025\nNote to self: ...
25524,Need to simulate a leak in Seal Injection Pipi...,Farley Maintenance,To facilitate training on a LOCA outside conta...,@djackson 01/15/2025\nAttaching updated nodal\...
25571,Condenser inleakage,Grand Gulf Simulator Modernization ENT22215,This was identified by LOR using the C25 LOR l...,@dofarrell 01/15/2025\nthe gist of this DR wit...
...,...,...,...,...
132,Data Request: Aux Bldg Ventilation system draw...,TMI Rad Transport EXE09083,Attached are additional drawings of the Aux an...,NaN
131,Create categories DR and Data Request,TMI Rad Transport EXE09083,Create a data request category so that this sy...,NaN
16,Cooper T & M Malf CS03 dtd 03/09/10,Cooper NPP08045,We have another work request for we would like...,@dofarrell 04/29/2010\nTalked to Don Dea - no ...
